In [64]:
# Learn to parse image!!

import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

CSV = 'animated_series.csv'
HOST = 'https://www.imdb.com'
URL = 'https://www.imdb.com/search/title/?genres=animation&title_type=tv_series&num_votes=1000,&sort=user_rating,desc'
# header needed so that the website doesn't think we are a crawler
HEADERS = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
}

def get_html(url, params=''):
    r = requests.get(url, headers=HEADERS, params=params)
    return r

def get_content(html):
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('div', class_='lister-item-content')
    kartoons = []
    
    for item in items:
        kartoons.append(
            {
                'title':item.find('h3', class_='lister-item-header').find('a').get_text(),
                'rating':item.find('div', class_='ratings-bar').find('strong').get_text(),
                'annotation':item.find_all('p', {'class':'text-muted'})[1].text.strip()
            }
        )
    return kartoons

def save_doc(items, path):
    with open(path, 'w', newline='') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(['Cartoon', 'Rank', 'Annotation'])
        for item in items:
            writer.writerow( [item['title'], item['rating'], item['annotation']] )

cartoons = []
def parser():
    PAGENATION = input('Specify the number of pages that you want to parse (from 1 to 20): ')
    PAGENATION = int(PAGENATION.strip())
    html = get_html(URL)
    if html.status_code == 200:
        for page in range(1, PAGENATION + 1):
            print(f'Parsing page: {page}')
            html = get_html(URL, params={'start': (50*(page-1) + 1)})
            cartoons.extend(get_content(html.text))
            #save_doc(cartoons, CSV)
        pass
    else:
        print('Error')

In [69]:
parser()

Specify the number of pages that you want to parse (from 1 to 20): 20
Parsing page: 1
Parsing page: 2
Parsing page: 3
Parsing page: 4
Parsing page: 5
Parsing page: 6
Parsing page: 7
Parsing page: 8
Parsing page: 9
Parsing page: 10
Parsing page: 11
Parsing page: 12
Parsing page: 13
Parsing page: 14
Parsing page: 15
Parsing page: 16
Parsing page: 17
Parsing page: 18
Parsing page: 19
Parsing page: 20


In [70]:
df3 = pd.DataFrame(cartoons)
df3

,title,rating,annotation
0,Bluey,9.7,The slice-of-life adventures of an Australian ...
1,Rick a Morty,9.2,An animated series that follows the exploits o...
2,Avatar: Legenda o Aangovi,9.2,"In a war-torn world of elemental magic, a youn..."
3,Celokovový Alchymista: Bratrství,9.1,Two brothers search for a Philosopher's Stone ...
4,Ginga eiyû densetsu,9.1,"Various people, especially two rising commande..."
...,...,...,...
1095,Noddy,5.7,Three children spend their time after school a...
1096,Super Drags,5.6,Three gay co-workers at a department store lea...
1097,Robot and Monster,5.6,"Two best friends, Robot and Monster, live toge..."
1098,Jake and the Never Land Pirates,5.6,An animated TV show that follows a band of you...
